In [1]:
import numpy as np
import pandas as pd
import os 
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_predict, KFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import gmean

from scipy.stats import norm

In [3]:
triploid_samples = ["8A697", "8A698", "8A699", "8A700", "8A701", "8A702", "8A703", 
                    "8A704", "8A711", "8A731", "8A732", "8A712"]  # "8A712",

diploid_samples = [
"8A681",
"8A682",
"8A689",
"8A690",
"8A691",
"8A692",
"8A693",
"8A694",
"8A695",
"8A696"
]


# Training DATA

In [4]:
def import_samples_using_copynumber(directory):
    """
    Imports data from CNV files in a specified directory, focuses on the 'copynumber' column,
    combines them into a single DataFrame, and adds a 'sample' column to identify the source of each file.

    Parameters:
        directory (str): The path to the directory containing CNV files.

    Returns:
        pd.DataFrame: A DataFrame containing the combined data focused on 'copynumber'.
    """
    combined_df = pd.DataFrame()  # Initialize an empty DataFrame
    lst = []  # To store filenames
    empty_files = []  # To store names of empty files
    failed_files = []  # To store names of files that failed to read
    duplicate_samples = []  # To store names of duplicate samples

    # Loop through all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.tsv'):  # Ensure the file is a .tsv file
            file_path = os.path.join(directory, filename)
            lst.append(filename)
            
            try:
                # Attempt to read the TSV file
                df = pd.read_csv(file_path, sep='\t')
                
                if df.empty:
                    empty_files.append(filename)  # Add empty file to the list
                    continue  # Skip the empty file
                
                # Strip any whitespace from the column names
                df.columns = df.columns.str.strip()
                
                # Add a column to identify the sample
                sample_name = filename.replace('_l1_cnv.tsv', '')
                df['sample'] = sample_name
                
                # Check if this sample is already in the combined DataFrame
                if 'sample' in combined_df.columns and sample_name in combined_df['sample'].unique():
                    duplicate_samples.append(sample_name)
                
                # Append the DataFrame to the combined DataFrame
                combined_df = pd.concat([combined_df, df], ignore_index=True)
            
            except Exception as e:
                # Add failed file to list and report error
                failed_files.append(filename)
                print(f"Error reading {filename}: {e}")
        if filename.endswith('sort_read_depth.csv'):  # Ensure the file is a .tsv file
            file_path = os.path.join(directory, filename)
            lst.append(filename)
            
            try:
                # Attempt to read the TSV file
                df = pd.read_csv(file_path)
                
                if df.empty:
                    empty_files.append(filename)  # Add empty file to the list
                    continue  # Skip the empty file
                
                # Strip any whitespace from the column names
                df.columns = df.columns.str.strip()
                
                # Add a column to identify the sample
                sample_name = filename.replace('_l1.sort_read_depth.csv', '')
                df['sample'] = sample_name
                
                
                # Check if this sample is already in the combined DataFrame
                if 'sample' in combined_df.columns and sample_name in combined_df['sample'].unique():
                    duplicate_samples.append(sample_name)
                
                # Append the DataFrame to the combined DataFrame
                combined_df = pd.concat([combined_df, df], ignore_index=True)
            
            except Exception as e:
                # Add failed file to list and report error
                failed_files.append(filename)
                print(f"Error reading {filename}: {e}")
    print(f"Total files processed: {len(lst)}")
    print(f"Total unique samples in DataFrame: {len(combined_df['sample'].unique()) if 'sample' in combined_df.columns else 0}")
    print(f"Empty files: {empty_files}")
    print(f"Failed to read files: {failed_files}")
    print(f"Duplicate samples: {duplicate_samples}")

    return combined_df

# Function to calculate geometric mean with handling for negative values
def safe_gmean(x):
    # Shift the values to make them all positive
    min_value = x.min()
    if min_value <= 0:
        shift_value = abs(min_value) + 1e-10  # Adding a small constant to ensure positivity
        x = x + shift_value  # Shift all values to positive
    return gmean(x)

def pivoting (combined_df,column_name = None): 
    # Step 1: Group by chromosome and sample, summing the 'copynumber' and other relevant columns
    try:
        grouped_df = combined_df.groupby(["chromosome", "sample"]).agg({
#             column_name:  lambda x: safe_gmean(x) # Apply the safe geometric mean function
              column_name:  "sum" # Apply the safe geometric mean function

        }).reset_index()
    except KeyError as e:
        print(f"KeyError in grouping: {e}")
        return None
    
    # Step 2: Pivot the DataFrame to make chromosomes as columns and samples as rows
    try:
        pivoted_df = grouped_df.pivot(index='sample', columns='chromosome', values=[column_name]).fillna(0)
    except Exception as e:
        print(f"Error during pivoting: {e}")
        return None
    
    # Flatten the multi-level column index after pivoting
    pivoted_df.columns = ['_'.join(col).strip() for col in pivoted_df.columns.values]
    
    # Step 3: Reset the index if you want 'sample' to be a column
    pivoted_df.reset_index(inplace=True)
    
    return pivoted_df

valid_samples = triploid_samples + diploid_samples

# Add labels to each DataFrame
def filter_and_label(df, sample_col="sample"):
    # Filter only valid samples
    filtered_df = df[df[sample_col].isin(valid_samples)].copy()
    
    # Add the 'sample_type' column based on the sample name
    filtered_df["sample_type"] = filtered_df[sample_col].apply(
        lambda x: "triploid" if x in triploid_samples else ("diploid" if x in diploid_samples
                                                           else ("haploid" if x in haploid_samples else 
                                                                 "unknown"))
    )
    return filtered_df



# Data to be Processed

In [ ]:
NY_K_dir = "/home/gaurav/Assignment/Triploidy_project/20241104_NY_K/triploidy classification/Working MOdel/copy/"

NY_K_dir_read_depth = '/home/gaurav/Assignment/Triploidy_project/20241104_NY_K/triploidy classification/Working MOdel/read/'

batch_cp = import_samples_using_copynumber(NY_K_dir)
batch_rd = import_samples_using_copynumber(NY_K_dir)

cp  = pivoting(batch_cp,column_name="copynumber")
rd  = pivoting(batch_rd,column_name="readcount")


merged_df = pd.merge(cp, rd, on=["sample"], suffixes=('_copynumber', 'readcount'))

merged_df["sample_type"] = merged_df['sample'].apply(
    lambda x: "triploid" if x in triploid_samples else ("diploid" if x in diploid_samples
                                                       else ("haploid" if x in haploid_samples else 
                                                             "unknown"))
)

merged_df = merged_df[merged_df["sample_type"]!="unknown"].reset_index(drop = True)

merged_filterd_df = merged_df.copy()

#  Performing the Training model : 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

# Step 1: Prepare the data
# Retain the 'sample' column for evaluation
samples = merged_filterd_df["sample"]

# Encode sample_type (target)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(merged_filterd_df["sample_type"])  # Encode target (diploid)

# Features (X)
chromosome_columns = [col for col in merged_filterd_df.columns if col.startswith(("read_depth","copynumber"))]
X = merged_filterd_df[chromosome_columns]  # Fixed order of chromosome features

# Step 2: Standardize the data (important for models like Logistic Regression, Gradient Boosting)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Scale the entire dataset


# Step 3: Train the model (use the entire dataset for training)
# No need for train-test split, we'll use the entire dataset
model = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
model.fit(X_scaled, y)

# Step 4: Evaluate the model on the same data (since you're training with the full dataset)
y_pred = model.predict(X_scaled)

# Step 5: Performance metrics
print(f"Model Classification Report:\n")
print(classification_report(y, y_pred, target_names=label_encoder.classes_))
print(f"Accuracy: {accuracy_score(y, y_pred):.2f}")

# Step 6: Attach predictions back to the dataset for sample-wise evaluation
predicted_sample_types = label_encoder.inverse_transform(y_pred)

# Combine results for sample-wise evaluation
results_df = X.copy()
results_df["sample"] = samples.reset_index(drop=True)
results_df["actual_sample_type"] = label_encoder.inverse_transform(y)
results_df["predicted_sample_type"] = predicted_sample_types

# Print the first few results
print(results_df.head())


# Saving the Model

In [8]:
import joblib
# Step 7: Save the model, scaler, and label encoder
model_dict = {
    'model': model,
    'scaler': scaler,
    'label_encoder': label_encoder
}

# Save the model and scaler as a dictionary
joblib.dump(model_dict, 'model_and_scaler_labelencoder_1.pkl')

print("Model, scaler, and label encoder saved successfully!")